<h1 style='font-size:40px'> Natural Language
Processing with RNNs and
Attention</h1>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O capítulo será composto por duas seções. Na primeira, iremos continuar desenvolvendo nossos estudos com RNN's, mas aplicadas no âmbito de NLP. Já na segunda, iremos dar enfoque aos mecanismos de atenção.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Generating Shakespearean Text Using a
Character RNN</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Aqui, vamos montar uma rede capaz de gerar poemas com o estilo de escrita de Shakespeare. Ela será treinada para prever o próximo caractere a ser digitado.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Creating the Training Dataset</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Antes de tudo, vamos baixar o corpus do projeto e associá-lo a uma variável.
        </li>
    </ul>
 </div>

In [6]:
# Baixando o arquivo com textos do Shakespeare. 
from tensorflow.keras.utils import get_file
file = "shakespeare.txt"
url = "https://homl.info/shakespeare"
filepath = get_file(file, url)

In [7]:
with open(filepath, 'r') as f:
    texts = f.read()

In [8]:
# Para numeralizarmos os textos, vamos recorrer à classe `Tokenizer`. Ao setarmos `char_level=True`, vamos associar cada caractere do corpus
# a um número (começando por 1). 
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([texts])

In [9]:
# Tokenização de caracteres.
tokenizer.texts_to_sequences(['Hi, there!'])

[[7, 6, 18, 1, 3, 7, 2, 9, 2, 31]]

In [10]:
# Observe que o objeto não considera caracteres acentuados, por padrão.
len(tokenizer.texts_to_sequences(['Olá, como vai?'])[0]), len('Ola, como vai?')

(13, 14)

In [11]:
# Convertendo índices em uma string.
tokenizer.sequences_to_texts([[1,2,3,4,5]])

['  e t o a']

In [12]:
# Atribuindo os índices do texto à variável encoded. Vamos fazer uma subtração para que o primeiro índice seja atribuído a 0.
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([texts])) - 1
encoded

array([19,  5,  8, ..., 20, 26, 10])

<h3 style='font-size:30px;font-style:italic'> How to Split a Sequential Dataset</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Vamos dividir aqui o nosso corpus no set de treino, validação e teste. O autor aproveita a situação, e revela os prós e contras das diferentes metodologias de separação do dataset. 
        </li>
        <li>
            No nosso caso, vamos optar por manter os primeiros fragmentos do corpus para treinamento, e o restante para validação e teste. Isso tem a assunção de que os mesmos padrões presentes em momentos anteriores do arquivo estarão contidos em momentos futuros (série estacionária).
        </li>
    </ul>
 </div>

In [13]:
from tensorflow.data import Dataset
train_size = encoded.shape[0] * 90 // 100
dataset = Dataset.from_tensor_slices(encoded[:train_size])

<h3 style='font-size:30px;font-style:italic'>Chopping the Sequential Dataset into Multiple Windows</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Até agora, convertemos todo o texto em uma única instância. Como treinar um modelo com isso é inviável, temos que quebrar esses dados em janelas, tratando cada uma delas como uma instância.
        </li>
        <li>
            A função `Dataset.window` cria datasets dentro de nosso dataset principal, cada um contendo uma porção específica de elementos. 
        </li>
    </ul>
 </div>

In [14]:
# Criando sub-datasets de 101 caracteres. Quando definimos `shift=1`, escolhemos que a janela se desloque 1 caractere por vez para montar 
# cada sub-dataset [0-101, 1-102...].

n_steps = 100
window_length = n_steps+1
# `drop_remainder` exclui as últimas janelas cujo tamanho acabaria menor do que `size`.
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

<div> 
    <ul style='font-size:20px'> 
        <li> 
            No entanto, devemos observar que as classes de modelo admitem apenas `tf.Tensor` como input. Caso quisermos converter cada Dataset aninhado em um tensor, podemos fazer um truque com a função `flat_map`, passando a ela uma lambda que cria batches de mesmo comprimento que `window_length`.
        </li>
        <li>
            Em tese, `flat_map` torna Datasets de aninhamento num único Dataset. Mas, como ele admite uma função que aplica alguma transformação nos Datasets aninhados antes da planificação, podemos aplicar `batch`. No final das contas, ficamos com tensores de mesmo conteúdo que os Datasets.
        </li>
    </ul>
 </div>

In [15]:
dataset = dataset.flat_map(lambda w: w.batch(window_length))

In [16]:
# Vamos aproveitar a situação, e já criar batches com vários chunks aleatórios.
dataset = dataset.shuffle(1000).batch(32)

In [17]:
# Criando o X e y da IA. Vamos programá-la para que ela receba a sentença recortada do primeiro ao penúltimo caractere,
# e preveja a sequência do segundo ao último caractere.
dataset = dataset.map(lambda w: (w[:, :-1], w[:, 1:]))

In [18]:
# Vamos tornar o X num One-Hot Encoding.
from tensorflow import one_hot

max_id = len(tokenizer.word_index) # Quantidade de caracteres distintos.
n = dataset.map(lambda X,y: (one_hot(X, depth=max_id), y)).as_numpy_iterator()

In [19]:
# Garantindo que o próximo batch a ser processado no treinamento já seja alocado à memória, antecipadamente.
dataset.prefetch(1)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

<h3 style='font-size:30px;font-style:italic'>Building and Training the Char-RNN Model</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Vamos montar agora uma rede recorrente que preverá o próximo caractere de uma sequência.
        </li>
        <li>
            Não montei a rede, porque demora muito para treinar.
        </li>
    </ul>
 </div>

In [20]:
import numpy as np
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

<h3 style='font-size:30px;font-style:italic'>Generating Fake Shakespearean Text</h3>
<div> 
    <ul style='font-size:20px'> 
        <li>
            A geração de textos completos poderia acontecer em ciclos, em que dada uma frase, o modelo prevê o próximo caractere e o vincula ao final da frase para uma nova previsão. O problema dessa abordagem é a alta probabilidade de sempre o mesmo caractere ser previsto...
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Temperature</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Uma maneira de garantirmos outputs mais diversos é acrescentar algumas etapas pós-predict do modelo. Tendo as probabilidades, extraímos o seu log e as dividimos por uma `temperature`. O valor desse argumento é de escolha do desenvolvedor.
        </li>
        <li>
            Nós usaremos a função `tf.random.categorical`, que escolherá o próximo token da frase, dado os logits computados.
        </li>
        <li>
            Menores temperatures ($<1$) acentuam as diferenças entre os logits, enquanto valores acima de 1 tendem a planificar esse conjunto de números. 
        </li>
    </ul>
 </div>

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [22]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

<h3 style='font-size:30px;font-style:italic'> Stateful RNN</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Uma RNN Stateful é aquela que preserva o seu state anterior para a próxima iteração de treinamento. No caso, isso é apenas recomendado caso a primeira instância do próximo batch seja uma continuação da última do batch anterior.
        </li>
        <li>
            No entanto, quando fazemos batch, as instâncias não são consecutivas por posição. No caso de $\text{batch}=32$, as primeiras instâncias dos dois primeiros batches (1 e 33) não são consecutivas.
        </li>
        <li>
            A solução para isso seria criar batches unitários!
        </li>
    </ul>
 </div>

In [ ]:
# Criando o dataset para uma RNN Stateful
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps,
drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id),
Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, TimeDistributed

# Deixe `stateful=True` em cada uma de suas camadas recorrentes.
# Informe `batch_input_shape` na primeira camada da rede.
stateful_model = Sequential([
    GRU(128, return_sequences=True, stateful=True, dropout=.2, recurrent_dropout=.2, batch_input_shape=[batch_size, None, max_id]),
    GRU(128, return_sequences=True, stateful=True, dropout=.2, recurrent_dropout=.2),
    TimeDistributed(Dense(max_id, activation='softmax'))
])

In [ ]:
# A única coisa que devemos nos atentar é resetar o state ao final de cada ÉPOCA.
# Para isso, vamos ter que criar um callback especializado nisso. Lembre-se de passá-lo em uma lista no `fit do modelo.
from tensorflow.keras.callbacks import Callback

class ResetStatesEpoch(Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

<h2 style='font-size:30px'> Sentiment Analysis</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Agora, vamos fazer um pequeno trabalho de análise de sentimentos com o IMDb. Um fato importante sobre ele é que seus textos já foram tokenizados por ordem de frequência - quanto menor o inteiro, mais frequente ele é.
        </li>
        <li>
            Os textos foram pré-processados antes da tokenização (lowercase, remoção de pontuação...).
        </li>
    </ul>
</div>

In [23]:
# Carregando as reviews.
from tensorflow.keras.datasets.imdb import load_data
(X_train, y_train), (X_test, y_test) = load_data()

17464789/17464789 [==============================] - 4s 0us/step


In [24]:
# Observe que os textos já estão tokenizados.
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [26]:
# Você pode decodificar o texto da seguinte maneira.
from tensorflow.keras.datasets.imdb import get_word_index
word_index =  get_word_index()

# Os 3 primeiros índices fazem referência a tokens especiais (padding, start-of-sequence e unknown words). Por isso, vamos ter que fazer um
# pequeno ajuste sobre as chaves.
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}

# Incluindo os tokens especiais ao nosso dicionário.
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
    
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

<h3 style='font-size:30px;font-style:italic'> Espaçamento entre Palavras</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Em projetos multilíngua, temos que considerar que nem todas as línguas separam palavras com espaço (chinês, vietnamita). Isso levou pesquisadores a fazer uma série de estudos sobre técnicas de tokenização de textos, como apresentado pelo livro.
        </li>
    </ul>
 </div>

<div> 
    <ul style='font-size:20px'> 
        <li> 
            O autor decide também elaborar um pequeno projeto do IMDb com as revisões em caracteres-byte.
        </li>
        <li>
            A solução vai se parecer com um CountVectorizer.
        </li>
    </ul>
 </div>

In [3]:
# Carregando o dataset.
from tensorflow_datasets import load
datasets, info = load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

/home/veiga/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-04 13:09:14.880683: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 13:09:15.443243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 13:09:15.443356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 13:09:15.533299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when 

Dl Completed...: 0 url [00:00, ? url/s]
Generating train examples...:   0%|      | 0/25000 [00:00<?, ? examples/s]
Generating train examples...:   0%| | 1/25000 [00:02<18:21:27,  2.64s/ exa
Generating train examples...:   8%| | 1936/25000 [00:03<00:33, 679.03 exam
Generating train examples...:  16%|▏| 3947/25000 [00:04<00:18, 1123.65 exa
Generating train examples...:  24%|▏| 5940/25000 [00:05<00:13, 1404.17 exa
Generating train examples...:  32%|▎| 7925/25000 [00:06<00:10, 1587.44 exa
Generating train examples...:  39%|▍| 9824/25000 [00:07<00:09, 1684.26 exa
Generating train examples...:  47%|▍| 11810/25000 [00:08<00:07, 1776.90 ex
Generating train examples...:  55%|▌| 13807/25000 [00:09<00:06, 1843.86 ex
Generating train examples...:  63%|▋| 15821/25000 [00:10<00:04, 1895.36 ex
Generating train examples...:  71%|▋| 17865/25000 [00:11<00:03, 1940.18 ex
Generating train examples...:  79%|▊| 19845/25000 [00:12<00:02, 1915.15 ex
Generating train examples...:  87%|▊| 21814/25000 [00:13<00:

Dataset imdb_reviews downloaded and prepared to /home/veiga/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Pequena função de encurtamento dos textos (questão de memória) e remoção das tags HTML.
# Também mantemos somente caracteres alfabéticos e dividimos as strings por espaçamento.
# Por fim, acrescentamos uma tag <pad> para garantir que todos os vetores sejam de mesmo tamanho.
from tensorflow.strings import substr, regex_replace, split

def preprocessing(X_batch, y_batch):
    X_batch = substr(X_batch, 0, 300)
    X_batch = regex_replace(X_batch, b'<br\\s*/?>', b' ')
    X_batch = regex_replace(X_batch, b'[^A-z\']', b' ')
    X_batch = split(X_batch)
    return X_batch.to_tensor(default_value=b'<pad>'), y_batch

In [ ]:
# Por questões de memória, vamos também manter apenas os 10000 tokens mais frequentes.
from collections import Counter
vocabulary = Counter()

for x,y in datasets['train'].batch(32).map(preprocessing):
    for review in x:
        vocabulary.update(list(review.numpy()))


vocab_size = 10000
truncated_vocab = [word for word, count in vocabulary.most_common(vocab_size)]

In [ ]:
# Inicializando nossa tabela texto-id.
from tensorflow import constant, int64
from tensorflow import range as tf_range
from tensorflow.lookup import KeyValueTensorInitializer, StaticVocabularyTable
num_oov_buckets = 1000
words = constant(truncated_vocab)
words_id = tf_range(len(words), dtype=int64)
vocab_init = KeyValueTensorInitializer(words, words_id)
table = StaticVocabularyTable(vocab_init, num_oov_buckets=num_oov_buckets)

In [ ]:
# Observe aqui que 'tripper' obteve um ID maior do que 10000, significando que não estava presente entre 
# os dados de treinamento.
table.lookup(constant([b'she was a day tripper one way ticket yeah'.split()]))

In [ ]:
# Função para codificação de textos.
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [ ]:
train_set = datasets['train'].batch(32).map(preprocessing)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

embed_size = 128
model = Sequential([
    Embedding(vocab_size + num_oov_buckets, embed_size,
    input_shape=[None]),
    GRU(128, return_sequences=True),
    GRU(128),
    Dense(1, activation="sigmoid")
])

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Embedding Layer</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            A Embedding Layer tem a tarefa de mapear uma palavra a um vetor. Sua ideia é conseguir encapsular a semântica do token nos componentes.
        </li>
        <li>
            Basicamente, ela consiste em uma matriz (n_classes, n_dimensoes). O que devemos fazer é recortar o vetor linha com o índice da classe sendo utilizada.
        </li>
        <li>
            Tendo o vetor em mãos, poderemos passá-lo como input de uma camada subsequente. Observe que nessa situação, faremos um forward pass por <strong>token</strong>. Isso fará com que a rede tenha que armazenar $\text{text-length}\times{\text{batch-size}}$ deriavadas parciais, antes do backpropagation.
        </li>
    </ul>
 </div>

<h3 style='font-size:30px;font-style:italic'> Masking</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O Keras ainda conta com uma funcionalidade de negligenciamento do token de padding (&lt;pad&gt;). Basicamente, a camada de Embedding lançará, junto com o vetor, um array booleano sinalizando se a camada seguinte deverá considerá-lo ou não.
        </li>
        <li>
            Essa máscara será propagada por todo o modelo, desde que a camada receptora lance um array mantendo a dimensão de tempo.
        </li>
    </ul>
 </div>

<h3 style='font-size:30px;font-style:italic'> Reusing Pretrained Embeddings</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Ao invés de treinarmos nossos próprios embeddings, podemos recorrer a componentes pré-treinados open-source. Eles estão disponíveis via TensorFlow Hub.
        </li>
        <li>
            Chamamos cada componente do TF-Hub como módulo.
        </li>
        <li>
            Aqui, testaremos a camada de Embedding nnlm-en-dim50. Ela medirá o vetor de cada token de sua string e retornará o embedding médio multiplicado pela raiz da quantidade de palavras do seu texto.
    <center style='margin-top:20px'>
        $\text{embedding}=\frac{\sqrt{n}}{n}\sum_{i=1}^{n}v_{i}$
    </center>
        </li>
    </ul>
 </div>

In [34]:
# Observe que os coeficientes dos módulos do TF-Hub vêm congelados. 
# Caso esteja interesado em fazer um fine-tuning em cima deles, defina o argumento `trainable=True`.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow_hub import KerasLayer
from tensorflow import string

model = Sequential([
        KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                        dtype=string, input_shape=[], output_shape=[50], trainable=True),
        Dense(128, activation="relu"),
        Dense(1, activation="sigmoid")
])

ModuleNotFoundError: No module named 'keras.src.engine.base_layer_v1'

In [ ]:
# Observe que o TF-Hub armazenará os módulos baixados no cache da sua máquina.
# Caso deseje que eles sejam depositados em disco, ponha o path desejado na variável de ambiente 
# 'TFHUB_CACHE_DIR'.
from os import environ
#environ['TFHUB_CACHE_DIR'] = '<path>'

<h2 style='font-size:30px'> An Encoder–Decoder Network for Neural Machine Translation</h2>
<div> 
    <ul style='font-size:20px'>
        <li>
            Nesta seção, aprendemos uma aplicação de RNN's Encoder-Decoder no cenário de tradução de textos (NMT). No paper, os autores empilham 4 LSTM's (acredito que tanto na parte encoder, quanto decoder). Ademais, eles descobriram que inverter a frase de input, antes de passá-la ao modelo, aprimora o desempenho em sequências longas!
        </li>
        <li> 
            Durante o treinamento, os pesquisadores buscaram fazer com que os batches tivessem sentenças de comprimento similares. Isso aprimorou a velocidade do treinamento em duas vezes!
        </li>
        <li>
            A geração da sentença é feita com um Beam Search.
        </li>
    </ul>
</div>

<figure>
    <center style='margin-top:20px'>
        <img src='16_sequence_to_sequence.png'>
            <figcaption> Arquitetura proposta pelo paper "Sequence-to-Sequence Learning"</figcaption>
    </center>
</figure>

<h3 style='font-size:30px;font-style:italic'> Beam Search</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Beam Search é uma técnica de geração de sequências. Tendo um modelo em mãos, pedimos para ele escolher os $\beta$ estados mais prováveis para serem o primeiro componente.
        </li>
        <li>
            Em seguida, pedimos para ele computar as probabilidades de transição $s_{1}\to{s_{2}}$, tendo como origem esses $\beta$ estados escolhidos. Novamente, selecionamos as $\beta$ transições mais prováveis - considerando todos os estados de origem em mãos, e não $\beta$ por estado. 
        </li>
        <li>
            Repetimos esse procedimento, até que todos os $\beta$ caminhos tenham atingido oo estado &lt;eos&gt;. Finalmente, escolhemos o caminho com a maior probabilidade!
        </li>
    </ul>
 </div>

<h3 style='font-size:30px;font-style:italic'> Outras Técnicas de Menção Válida</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Géron diz que providenciar a palavra anterior na previsão da próxima pode facilitar o treinamento da rede. Em treinamento, insira a palavra que deveria ter sido prevista e, em inferência, a que foi prevista na última iteração.
            <center style='margin-top:20px'>
                <img src='16_previous_token.png'>
            </center>
        </li>
        <li style='margin-top:20px'>
            Computar a probabilidade de cada token do vocabulário de output pode onerar desnecessariamente a nossa máquina. Por isso, em treinamento, costuma-se medir a softmax de apenas o token da target junto com uma amostra aleatória. Em inferência, vamos ter que recorrer à softmax padrão mesmo.
        </li>
    </ul>
 </div>

<h3 style='font-size:30px;font-style:italic'> Bidirectional RNNs</h3>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Nas arquiteturas vistas até então, vetorizamos tokens levando em conta apenas as palavras que os antecedem. Muitas vezes, porém, considerar tokens que o sucedem podem fornecer uma vetorização mais contextualizada. Para essa tarefa, foram criadas as RNN's bidirecionais. 
        </li>
        <li> 
            Nela, cada célula da rede ganhará uma irmã-gêmea, encarregada de processar o mesmo input. No entanto, ela o lerá de trás para frente.  
        </li>
        <li>
            No final, o output desse par será o concatenado dos seus dois vetores de resultado. Portanto, se as camadas lançam vetores de $n$ dimensões, o output oficial será de $2\times{n}$.
              <center style='margin-top:20px'>
                <img src='bidirectional_rnn.png'>
            </center>
        </li>
    </ul>
 </div>

<h2 style='font-size:30px'> Attention Mechanisms</h2>
<div> 
    <ul style='font-size:20px'>
        <li>
            As técnicas de tradução de textos vistas até agora têm bastante dificuldade em traduzir textos compridos. No caso do modelo do "Sequence-to-Sequence Learning", observe que os hidden states de palavras mais afastadas do começo são "sobrepostos" pelos das palavras iniciais. 
        </li>
        <li>
            A fim de lidar com messe problema, o paper 'Neural Machine Translation By Jointly Learning To Align And Translate' propõe uma nova arquitetura de modelos de tradução, recorrendo a RNN's Bidirecionais (BiRNN).
        </li>
        <li>
            Com seu uso, cada palavra receberá um hidden state próprio, levando em conta as palavras que a precedem e sucedem. Nessa situação não precisaremos mais nos preocupar com a sobreposição dos estados.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Additive/Concat/Bahdanau Attention</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Tendo esses hidden states ($h$) em mãos, os passamos a um MLP, juntamente com o state da última previsão do decoder ($s_{t-1}$) para a computação de um score $e$.
        </li>
        <li>
            Esses scores/energias do MLP passam por uma softmax, que nos retorna valores $\alpha$ indicando o grau de relação de cada palavra com a última previsão do decoder.
        </li>
        <li>
            Por fim, computamos um vetor $c$ com base na soma dos hidden states, ponderados pelos seus respectivos $\alpha$'s. O decoder o receberá, juntamente com o hidden state e target da iteração anterior para a computação da previsão.
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Benefício do Mecanismo de Atenção</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            O vetor $c$ possibilita uma contextualização mais precisa ao decoder sobre quais informações da frase de origem ele deverá se <i> atentar</i> mais. Em arquiteturas anteriores, o uso de um hidden state imutável fazia o decoder considerar todo o conteúdo do texto - incluindo aquele que não era necessário - na previsão de cada token.
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Computações Encoder</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Sendo $T_x$ a quantidade de palavras da sentença origem, $j$ o indexador de uma palavra da sentença de origem, e $i$ o de uma outra palavra da frase sendo prevista:
            <center style='margin-top:20px'>
                $e_{ij}=v_{a}^{T}\tanh{(W_a[s_{i-1};h_{j}])}$
            </center>
            <center style='margin-top:20px'>
                $\alpha_{ij}=\frac{\exp(e_{ij})}{\sum_{k=1}^{T_x}\exp(e_{ik})}$
            </center>
            <center style='margin-top:20px'>
                $c_i=\displaystyle \sum_{j=1}^{T_x}\alpha_{ij}h_j$
            </center>
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Computações Decoder</h4>
<div style='font-size:20px'> 
    <center style='margin-top:20px'>
        $s_i=f(s_{i-1},y_{i-1},c_i)$
    </center>
    <center style='margin-top:20px'>
        $p(y_i|y_1,...,y_{i-1},x)=g(y_{i-1},s_i,c_i)$
    </center>
 </div>

<h3 style='font-size:30px;font-style:italic'> Dot/Luong Attention</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O paper "Effective Approaches to Attention-based Neural Machine Translation" propõe uma computação mais simplificada das energias de cada palavra (dot-product). São apresentadas duas modalidades para essa conta.
            <center style='margin-top:20px'>
                $$
                    e_{ij}=
                        \begin{cases}
                            s_{i}^{T}h_{j}  &\text{(dot)} \\
                            s_{i}^{T}Wh_{j}  &\text{(general)}
                        \end{cases}
                $$
            </center>
        </li>
        <li style='margin-top:20px'>
            É importante frisar que o dot-attention recorre ao state <strong> atual</strong> do decoder na computação das energias. Além disso, o vetor de contexto $c_i$ será computado pela média dos hidden states das palavras ponderada pelos coeficientes gerados pela softmax. 
            <center style='margin-top:20px'>
                $\displaystyle c_i=\frac{1}{T_x}\sum_{i=1}^{T_x}\alpha_{ij}h_{j}$
            </center>
        </li>
        <li style='margin-top:20px'>
            Assim como em Bahdanau, todas as palavras são levadas em conta na computação de $c_i$, por padrão. Essa abordagem é nomeada pelos autores como Atenção Global. A fim de reduzir o custo computacional dessa operação, o paper propôs a chamada Local Attention.
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Local Attention</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            A Local Attention propõe criar um certo $c_i$, considerando apenas um intervalo $[p_i-D,p_i+D]$ da frase-fonte ($D$ é um inteiro pré-determinado).
        </li>
        <li>
            A posição central $p_i$ pode ser simplesmente $i$, considerando que a ordenação das palavras nas frases de origem e target seja a mesma. Por outro lado, os autores também propõe uma definição um pouco mais sofisticada:
            <center style='margin-top:20px'>
                $p_i=T_x \times{\text{sigmoid}{(v_{p}^{T}\tanh{(W_{p}s_{i})})}}$
            </center>
            <p style='font-size:10px;margin-left:15%'> 
                    $v_p$ e $W_p$ terão seus coeficientes aprendidos no decorrer do treinamento.
                </p>
        </li>
        <li style='margin-top:20px'>
            Os coeficientes da média ponderada serão calculados da seguinte forma:
             <center style='margin-top:20px'>
                $\displaystyle \alpha_{ij}=\frac{\exp(e_{ij})}{\sum_{k\in{[p_i-D,p_i+D]}}\exp(e_{ik})}\times \exp{(-\frac{(j-p_i)^{2}}{2(\frac{D}{2})^{2}})}$
            <center style='margin-top:20px'>
            </center>
            </center>
        </li>
    </ul>
 </div>

In [ ]:
# https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html (Self-Attention)

In [6]:
! git add .
! git commit -am 'Continuar Attention is All You Need (Multi-Head Attention)'
! git push

[master 97dac8b] Continuar Attention is All You Need (Multi-Head Attention)
 2 files changed, 10 insertions(+), 19 deletions(-)
Enumerating objects: 11, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 24 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 650 bytes | 650.00 KiB/s, done.
Total 6 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 4 local objects.
To https://github.com/felipesveiga/Hands-On-Machine-Learning.git
   a9e8618..97dac8b  master -> master


<p style='color:red'>  Vi de Positional Encoding até Training; Continuar Attention is All You Need (Results)</p>